# Causal Effects via Regression

Let's take up the example from the slides: what is the effect of going to a fancy college on later-life earnings? We'll use data on about 1,000 American men in the NLSY born 1980-1984 who finished college, and look at the effect of going to a private college ($D_i$) on earnings ($Y_i$) in 2015-2019 (when they were about 30-39 years old). We will be estimating an equation like this:
$$
Y_i = \delta D_i + X_i'\beta+\varepsilon_i,
$$
where $X_i$ is a vector of controls, conditional on which we are willing to assume $D_i$ is as good as randomly assigned.

What kinds of variables should we include in $X_i$?


In [ ]:
# import useful packages
import pandas as pd # for loading and managing datasets
import statsmodels.api as sm # for running regressions and getting standard errors

In [ ]:
# load NLSY data
nlsy = pd.read_csv("https://www.dropbox.com/s/2csnuyk7xlsywkh/nlsy97.csv?raw=true")
nlsy

In [ ]:
# clean data (drop obs with missing values)
nlsy = nlsy.dropna()
nlsy

Let's start with a simple (uncontrolled) regression.

In [ ]:
# Simple regression
rhs = sm.add_constant(nlsy['privatecollege']) # you have to add the constant yourself with statsmodels!
model = sm.OLS(nlsy['annualearnings'], rhs)
results = model.fit(cov_type='HC3') # heteroskedasticity-robust
print(results.summary())

How to interpret the coefficient on $privatecollege$? As a causal effect?

Now let's add controls for parent's education and cognitive ability as measured by ASVAB:

In [ ]:
# Regression with controls
rhs = sm.add_constant(nlsy[['privatecollege','dadcollege','momcollege','asvab']])
model = sm.OLS(nlsy['annualearnings'], rhs)
results = model.fit(cov_type='HC3')
print(results.summary())

How did the inclusion of controls change the estimate? Why?

Back to the whiteboard for prediction!

# Prediction Primer

Let's use decision trees to predict which participants of the National JTPA Study were likely to find a job. We will use prior earnings, education, sex, race, and marital status as our prediction features.

In [ ]:
data=pd.read_csv("https://www.dropbox.com/s/jgu51n2kc72cp5j/jtpahet.csv?raw=true")
data

Import some utilities:

In [ ]:
#@title
import requests
url1 = 'https://www.dropbox.com/s/jgml061manxpawo/plot_2d_separator.py?raw=true'
url2 = 'https://www.dropbox.com/s/hlrrlwm4kt36awb/plot_interactive_tree.py?raw=true'
url3 = 'https://www.dropbox.com/s/e2cy203sr30a59z/plot_helpers.py?raw=true'
url4 = 'https://www.dropbox.com/s/aik5sgcwgz4brwn/tools.py?raw=true'
r1 = requests.get(url1)
r2 = requests.get(url2)
r3 = requests.get(url3)
r4 = requests.get(url4)

# make sure your filename is the same as how you want to import 
with open('plot_2d_separator.py', 'w') as f1:
    f1.write(r1.text)

with open('plot_interactive_tree.py', 'w') as f2:
    f2.write(r2.text)

with open('plot_helpers.py', 'w') as f3:
    f3.write(r3.text)

with open('tools.py', 'w') as f4:
    f4.write(r4.text)

# now we can import
import plot_helpers
import tools
import plot_2d_separator
import plot_interactive_tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from sklearn.tree import plot_tree

We'll first grow a tree using just two features (education and prior earnings) so we can visualize it easily. Let's visualize the feature space: triangles are individuals who found a job, circles are those who didn't.

In [ ]:
plot_helpers.discrete_scatter(data.loc[:,'educ'].values, data.loc[:,'priorearn'].values, data.loc[:,'foundjob'].values)
plt.show()

In [ ]:
tree = DecisionTreeRegressor(max_depth=3).fit(data.loc[:,['educ','priorearn']].values,data.loc[:,'foundjob'].values)
fig1,ax = plt.subplots(1,1,figsize=(12, 8))
plot_interactive_tree.plot_tree_partition(data.loc[:,['educ','priorearn']].values, data.loc[:,'foundjob'].values, tree, ax=ax)
plot_tree(tree, feature_names=['education','Prior earnings'],class_names=['No job','Found job'], impurity=False, filled=True)
plt.show()

Now let's do a random forest:

In [ ]:
forest = RandomForestClassifier(n_estimators=5, random_state=2).fit(data.loc[:,['educ','priorearn']].values,data.loc[:,'foundjob'].values)

fig, axes = plt.subplots(2, 3, figsize=(20, 10))
for i, (ax, tree) in enumerate(zip(axes.ravel(), forest.estimators_)):
    ax.set_title("Tree {}".format(i))
    plot_interactive_tree.plot_tree_partition(data.loc[:,['educ','priorearn']].values,data.loc[:,'foundjob'].values, tree, ax=ax)
    
plot_2d_separator.plot_2d_separator(forest, data.loc[:,['educ','priorearn']].values, fill=True, ax=axes[-1, -1],
                                alpha=.4)
axes[-1, -1].set_title("Random Forest")
plot_helpers.discrete_scatter(data.loc[:,'educ'].values, data.loc[:,'priorearn'].values, data.loc[:,'foundjob'].values)
plt.show()

We only used two prediction features (prior earnings and education) for visualization. To get the best predictions, we should use all of our features. And to evaluate the quality of the prediction, we should hold out a test set.

In [ ]:
# Define a matrix of features:
X = data[['age','priorearn','educ','female','nonwhite','married']]
# hold out a test test:
X_train, X_test, y_train, y_test = train_test_split(X, data['foundjob'], random_state=42)

Try on your own: grow a forest with 500 trees using the training set, and evaluate the prediction accuracy on the test set. Hint: you can evaluate the prediction accuracy by doing `forest.score(X_test,y_test)`.

##Cheat

In [ ]:
forest = RandomForestClassifier(n_estimators=500, random_state=2).fit(X_train,y_train)
forest.score(X_test,y_test)

So much for predicting *outcomes*. We want to predict causal effects. Back to the whiteboard!

# Using Machine Learning to Predict Heterogeneous Treatment Effects

## Key Challenge: Algorithms tailored for predicting outcomes can do poorly when predicting treatment effects

### Factors that strongly predict outcomes may not strongly predict treatment effects

$Y_i$: spending on a Lexus

$D_i$: seeing an online ad for a Lexus

$\ln Y_i=\beta_0+\beta_1 age_i +\beta_2 male_i + \beta_3 D_i+\beta_4 D_i \times male_i +\varepsilon_i$

How do outcomes vary by age? (A lot if $\beta_1$ is big)

How do treatment effects vary by age? (not at all!)

What do treatment effects vary by? (gender!)

Let's simulate some data to show what happens when we try to use algorithm tailored to predicting outcomes for predicting treatment effects.

In [ ]:
import numpy as np
from sklearn import tree

In [ ]:
# define parameters
n = 1000 # sample size
p = .5 # probability of seeing the ad
beta0=0
beta1=.2 # effect of age
beta2=-.025 # difference in average spending between males and females who don't see the ad ()
beta3=0 # effect of treatment among females
beta4=.05 # differential effect of treatment among males compared to females
sigeps=.02 # residual variance of outcome

# generate some fake data
age=np.random.randint(low=18,high=61,size=(n,1))
male=np.random.randint(low=0,high=2,size=(n,1))
d=np.random.rand(n,1)>(1-p)
epsilon=sigeps*np.random.randn(n,1)
lny=beta0+beta1*age+beta2*male+beta3*d+beta4*d*male+epsilon

# assemble as dataframe
fakedata=pd.DataFrame(np.concatenate((lny,d,age,male),axis=1),columns=['lny','d','age','male'])
fakedata.feature_names=['age','male']
x0 = fakedata.loc[d==0,['age','male']]
x1 = fakedata.loc[d==1,['age','male']]
y0 = fakedata.loc[d==0,['lny']]
y1 = fakedata.loc[d==1,['lny']]

Try on your own: fit two trees (call them `tree0` and `tree1`), each with `max_depth=2` to predict the outcome separately in the untreated ($D_i=0$) and treated ($D_i=1$) samples, using `x0` and `x1`, respectively.

In [ ]:
# fit trees

In [ ]:
# display trees

### Cheat

In [ ]:
# fit trees
tree1 = DecisionTreeRegressor(max_depth=2).fit(x1,y1)
tree0 = DecisionTreeRegressor(max_depth=2).fit(x0,y0)

In [ ]:
# display trees
print('Treated tree:')
plot_tree(tree1,filled=True,feature_names=fakedata.feature_names) 
plt.show()

In [ ]:
print('Untreated tree:')
plot_tree(tree0,filled=True,feature_names=fakedata.feature_names) 
plt.show()

Which variable(s) did the trees key in on? Why? Would these trees be useful for predicting treatment effects? Why or why not?

How do we fix the problem?

## Random Causal Forest: Simulated Example

In [ ]:
# Install econml
!pip install econml #&> /dev/null
from econml.dml import CausalForestDML as CausalForest

In [ ]:
# Instantiate the Causal Forest
estimator = CausalForest(n_estimators=500,discrete_treatment=True,criterion="het")
# Grow the forest
estimator.fit(fakedata['lny'], # outcoe
              fakedata['d'], # treatment
              X=fakedata[['age','male']]) # prediction features
# Predict effects for each observation based on its characteristics:
effects = estimator.effect(fakedata[['age','male']])

Let's see how well it did at estimating effects among men and women:

In [ ]:
malefx=effects[fakedata['male'].values==1]
malefx.mean()




In [ ]:
femalefx=effects[fakedata['male'].values==0]
femalefx.mean()

How did our causal forest do at getting effects right for men and women?
Let's see how it does on the age profile:

In [ ]:
fig = plt.figure()
ax = plt.axes()
maleage=fakedata['age'].iloc[fakedata['male'].values==1]
femaleage=fakedata['age'].iloc[fakedata['male'].values==0]

ax.scatter(maleage,malefx,label='males');
ax.scatter(femaleage,femalefx,label='females');
ax.legend()
plt.title("Estimated Treatment effects")
plt.xlabel("age")
plt.ylabel("treatment effect");

A little noisy on the age profile (which should be flat) but does get the difference between men and women!

## Random Causal Forest: Predict the effects of job training

We are ready to apply machine learning to predict causal effects in a real-life setting: how do the effects of job training vary by an individual's characteristics? We will use data from the National Job Training Partnership study, a large-scale randomized evaluation of a publicly subsidized job training program for disadvantaged youth and young adults. Why would we care how the effects of a subsidized job training program vary by a person's characteristics?

We will use the JTPA evaluation dataset, which contains observations on about 14,000 individuals, some of whom were randomized to participate in job training ($z_i = 1$) and others who were not ($z_i = 0$).

To do on your own:


*   load the dataset from the url `https://www.dropbox.com/s/jgu51n2kc72cp5j/jtpahet.csv?raw=true`
*   define the outcome vector (call it `y`) to be the column labeled `foundjob`
*   define the randomized assignment indicator (call it `z`) to be the column labeled `z`
*   define the feature vector (call it `x`) to be all columns except `foundjob`, `z`, and `enroll`.



In [ ]:
# load the data

# define the variables


### Cheat

In [ ]:
data=pd.read_csv("https://www.dropbox.com/s/jgu51n2kc72cp5j/jtpahet.csv?raw=true")
data

In [ ]:
y = data['foundjob']
z = data['z']
x = data.drop(['foundjob','z','enroll'],axis=1)
x

### Regression to get average effect

On your own: run a linear regression of the outcome on the random assignment indicator, `z`. Since this was a randomized experiment, we don't need controls!

### Cheat:

In [ ]:
rhs = sm.add_constant(data['z']) # you have to add the constant yourself with statsmodels!
model = sm.OLS(data['foundjob'], rhs)
results = model.fit(cov_type='HC3') # heteroskedasticity-robust
print(results.summary())

### Set up random forest

So far, so good? Now create a random causal forest object, and fit it with outcome `y`, treatment variable `z`, and feature matrix `x`.

In [ ]:
# On your own: create and fit random causal forest object

### Cheat

In [ ]:
rcf = CausalForest(n_estimators=500,discrete_treatment=True,criterion="het").fit(y,z,X=x)


### Explore effects

Let's see what kind of heterogeneous effects our random causal forest predicted

In [ ]:
# calculate the predicted effects:
insamplefx =rcf.effect(x)
# plot a histogram of the estimated effects, with average effect overlaid
fig=plt.figure()
ax = plt.axes()
ax.hist(insamplefx,bins=30,density=True)
plt.axvline(rcf.ate_, color='k', linestyle='dashed', linewidth=1)
plt.suptitle("Estimated Treatment effects")
plt.title("ATE: {:.3g}".format(rcf.ate_[0]))
plt.show()

Let's visualize how these effects vary by prior earnings and education by making a heatmap

In [ ]:
import itertools

In [ ]:
# create a grid of values for education and prior earnings:
educgrid=np.arange(data['educ'].values.min(),data['educ'].values.max()+1)
earngrid = np.arange(data['priorearn'].values.min(),data['priorearn'].values.max(),5000)
grid = pd.DataFrame(itertools.product(educgrid,earngrid),columns=['educ','priorearn'])


We'll first visualize the effects among married, nonwhite females of average age:

In [ ]:
grid['age']=data['age'].values.mean() # set age to the average
grid['female']=1 # set female = 1
grid['nonwhite']=1 # set nonwhite = 1
grid['married']=1 # set married = 1


To do on your own: calculate the predicted effects for each "observation" in the grid:

In [ ]:
# gridfx = # uncomment and fill in on your own!

### Cheat

In [ ]:
gridfx = rcf.effect(grid)

### Visualize effects with a heatmap:

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig = plt.figure()
ax = plt.subplot()
main=ax.scatter(grid['educ'],grid['priorearn'],c=gridfx,cmap='plasma',marker="s",s=300);
plt.suptitle("Estimated Treatment effects")
plt.title("Nonwhite married females")
plt.xlabel("years of education")
plt.ylabel("prior earnings");

# create an Axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(main, cax=cax)
plt.show()

To do on your own: make similar visualizations for males, singles, whites, different ages, etc.